In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# We assume you uploaded the exercise folder in root Google Drive folder
!cp -r /content/drive/MyDrive/3d-lmnet-pytorch 3d-lmnet-pytorch
os.chdir('/content/drive/MyDrive/3d-lmnet-pytorch')
print('Installing requirements')
!pip install -r requirements.txt

# Make sure you restart runtime when directed by Colab

In [19]:
import os
import sys
import torch
os.chdir('/content/3d-lmnet-pytorch/3d-lmnet-pytorch')
sys.path.insert(1, "/content/3d-lmnet-pytorch/3d-lmnet-pytorch")
print('CUDA availability:', torch.cuda.is_available())

CUDA availability: True


In [20]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
torch.cuda.is_available()

True

## ShapeNet Terms and Conditions

In order to be able to use the data, we agree the below terms and conditions:

1. Researcher shall use the Database only for non-commercial research and educational purposes.
2. Princeton University and Stanford University make no representations or warranties regarding the Database, including but not limited to warranties of non-infringement or fitness for a particular purpose.
3. Researcher accepts full responsibility for his or her use of the Database and shall defend and indemnify Princeton University and Stanford University, including their employees, Trustees, officers and agents, against any and all claims arising from Researcher's use of the Database, including but not limited to Researcher's use of any copies of copyrighted 3D models that he or she may create from the Database.
4. Researcher may provide research associates and colleagues with access to the Database provided that they first agree to be bound by these terms and conditions.
5. Princeton University and Stanford University reserve the right to terminate Researcher's access to the Database at any time.
6. If Researcher is employed by a for-profit, commercial entity, Researcher's employer shall also be bound by these terms and conditions, and Researcher hereby represents that he or she is fully authorized to enter into this agreement on behalf of such employer.
7. The law of the State of New Jersey shall apply to all disputes under this agreement.

### Unzip ShapeNet pointcloud zip

In [ ]:
!unzip -q ./data/ShapeNet_pointclouds.zip -d ./data

### Download 2D images

In [ ]:
!wget http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz -P ./data

--2022-12-17 23:31:14--  http://cvgl.stanford.edu/data2/ShapeNetRendering.tgz
Resolving cvgl.stanford.edu (cvgl.stanford.edu)... 171.64.64.64
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cvgl.stanford.edu/data2/ShapeNetRendering.tgz [following]
--2022-12-17 23:31:14--  https://cvgl.stanford.edu/data2/ShapeNetRendering.tgz
Connecting to cvgl.stanford.edu (cvgl.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12318245442 (11G) [application/x-gzip]
Saving to: ‘/content/term_project/data/ShapeNetRendering.tgz’

ShapeNetRendering.t 100%[===================>]  11.47G  13.1MB/s    in 13m 8s  

2022-12-17 23:44:23 (14.9 MB/s) - ‘/content/term_project/data/ShapeNetRendering.tgz’ saved [12318245442/12318245442]



In [ ]:
!tar -xf ./data/ShapeNetRendering.tgz -C ./data
#!rm /content/term-project/data/ShapeNetRendering.tgz

### Construct ShapeNet dataset

In [ ]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train')
val_dataset = ShapeNet('valid')
#overfit_dataset = ShapeNet('overfit')
test_dataset = ShapeNet('test')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}') 
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(val_dataset)}') 
# Get length, which is a call to __len__ function
print(f'Length of test set: {len(test_dataset)}')  

Length of train set: 26271
Length of val set: 8758
Length of test set: 8755


In [22]:

from skimage.measure import marching_cubes

train_sample = train_dataset[1]
print(f'Input images: {train_sample["img"].shape}')  
print(f'Input point cloud: {train_sample["point"].shape}')  

Input images: (24, 137, 3, 137)
Input point cloud: (2048, 3)


### Print output shape of the 2D Encoder model (both variational and normal versions)

In [ ]:
from model.model_2d import ImageEncoder

model2d_variational=ImageEncoder("variational",512)

input_tensor = torch.randn(137,137,3)
mu,std = model2d_variational(input_tensor)
print("Mu:",mu,"Std:",std)

model2d_normal=ImageEncoder("normal",512)

latent=model2d_normal(input_tensor)
print("Latent shape:",latent.shape)

ValueError: ignored

### Train 2D Encoder model to match the predicted latent space to the output of 3D Encoder of pointclouds

In [ ]:
from training import train_2d_to_3d

generalization_config = {
    'experiment_name': '2d_to_3d_variational',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'bottleneck': 512,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 30,  
    'print_every_n': 5,
    'visualize_every_n': 5,
}

model2d_variational.main(generalization_config)

NameError: ignored

### Infer pointclouds using trained 2D Encoder and 3D Decoder models

Variational inferences:

In [ ]:
from inference.infer_2d_to_3d import Inference2DToPointCloudVariational

device = torch.device('cuda:0')  # change this to cpu if you're not using a gpu
id=torch.randint(0,len(test_dataset))
val_config={"final_layer" : "variational",
        "bottleneck" : 512,
        "input_size" : None,
        "hidden_size" : None,
        "output_size" : None,
        "bnorm" : True,
        "bnorm_final" : False,
        "regularizer" : None,
        "weight_decay" : 0.001,
        "dropout_prob" : None}
Inference2DToPointCloudVariational(test_dataset[id],"content/3d-lmnet-pytorch/runs/2d_to_3d_variational","content/3d-lmnet-pytorch/runs/3d_pointcloud_decoder", val_config,device)

In [ ]:
predicted_point_clouds=inference_handler_variational.infer()

Normal inferences:

In [ ]:
from inference.infer_2d_to_3d import Inference2DToPointCloudNormal

id=torch.randint(0,len(test_dataset))
val_config={"final_layer" : "normal",
        "bottleneck" : 512,
        "input_size" : None,
        "hidden_size" : None,
        "output_size" : None,
        "bnorm" : True,
        "bnorm_final" : False,
        "regularizer" : None,
        "weight_decay" : 0.001,
        "dropout_prob" : None}
Inference2DToPointCloudNormal(test_dataset[id],"content/3d-lmnet-pytorch/runs/2d_to_3d_normal", "content/3d-lmnet-pytorch/runs/3d_pointcloud_decoder",val_config,device)

In [ ]:
predicted_point_cloud=inference_handler_normal.infer()

In [ ]:
from training import train_ae


    # parser.add_argument("--root", type=str, default="./data")
    # parser.add_argument("--npoints", type=int, default=2048)
    # parser.add_argument("--mpoints", type=int, default=2025)
    # parser.add_argument("--batch_size", type=int, default=16)
    # parser.add_argument("--lr", type=float, default=1e-4)
    # parser.add_argument("--weight_decay", type=float, default=1e-6)
    # parser.add_argument("--epochs", type=int, default=400)
    # parser.add_argument("--num_workers", type=int, default=4)
    # parser.add_argument("--log_dir", type=str, default="./log")

generalization_config = {
    'root': './3d-lmnet-pytorch/',
    'experiment_name': '3d_autoencoder',
    'device': 'cuda:0',  # run this on a gpu for a reasonable training time
    'is_overfit': False,
    'npoints': 2048,
    'mpoints': 2025,
    'lr': 1e-4,
    'weight_decay': 1e-6,
    'bottleneck': 512,
    'batch_size': 16,
    'resume_ckpt': None,
    'learning_rate_model':  0.00005,
    'max_epochs': 400,  
    'num_workers': 4,
    "input_size" : 256,
    "hidden_size" : 256,
    "output_size" : 256,
    'log_dir': './3d-lmnet-pytorch/',
    'print_every_n': 5,
    'visualize_every_n': 5,
}

train_ae.main(generalization_config)

TypeError: ignored